In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio
import pyimpute
import dask.dataframe as dd
from shapely.geometry import Point
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection 
from sklearn import metrics
from classes.domain import Domain
from pyimpute import load_training_vector, load_targets, impute, evaluate_clf
import plotly.express as px
import plotly.graph_objects as go
import json


## 1 Tratando os dados de ocorrências

### Etapas
- Filtrando somente as observações no Brasil
- Removendo dados que não contém informações de data
- Padronizando o nome dos estados
- Padronizando a contagem como 1 para dados que contém apenas as ocorrencias
- Criando um arquivo com as ocorrências de latitude e longitude de ocorrências

### Saída:

Arquivo gerado: ``assets/INPUT/occurence.parquet``

Tabela gerada:
| Latitude | Longitude | Presence |
|----------|-----------|----------|
|   -30    |    -30    |    1     |
|   -28    |    -24    |    1     |
|   -20    |    -31    |    1     |
|   ...    |    ...    |   ...    |

In [ ]:
GBIF_RAW_FILE = "assets/INPUT/gbif.csv"
GBIF_TREATED_FILE_OUTPUT = "assets/INPUT/gbif.parquet"
OCCURANCE_FILE = "assets/INPUT/occurence.parquet"
OCCURANCE_ABSENCE_FILE = "assets/INPUT/occurence_abscence.parquet"
N_PSEUDO_ABSCENCE = 9_000

STATE_NAME_ENCODING = {
   "Santa Catarina":"SC",
   "São Paulo":"SP",
   "Rio Grande do Sul":"RS",
   "Minas Gerais":"MG",
   "Paraná":"PR",
   "Espírito Santo":"ES",
   "Brazil - São Paulo":"SP",
   "Rio de Janeiro":"RJ",
   "Brazil - Minas Gerais":"MG",
   "Bahia":"BA",
   "Mato Grosso do Sul":"MS",
   "Parana":"PR",
   "Brazil - Santa Catarina":"SC",
   "Sp":"SP"  
}

dtype_dict = {
        'dateIdentified': 'object',
       'day': 'float64',
       'establishmentMeans': 'object',
       'identifiedBy': 'object',
       'mediaType': 'object',
       'month': 'float64',
       'recordNumber': 'object',
       'rightsHolder': 'object',
       'verbatimScientificNameAuthorship': 'object',
       'year': 'float64'
}


In [ ]:
## Selecting and transforming data to correct data types
occurence_species_data_unfiltered = dd.read_csv(
       GBIF_RAW_FILE,
       sep='\t',
       dtype=dtype_dict
).reset_index().compute().to_parquet(GBIF_TREATED_FILE_OUTPUT)

occurence_species_data = pd.read_parquet(
    GBIF_TREATED_FILE_OUTPUT, 
    columns=[
        'countryCode',
        'locality',
        'decimalLatitude',
        'decimalLongitude',
        'eventDate',
        'individualCount',
        'basisOfRecord',
        'collectionCode',
        'stateProvince'],
).reset_index(drop=True)

occurence_species_data = occurence_species_data[occurence_species_data['countryCode'] == 'BR'] # Filtering only in Brazil
occurence_species_data = occurence_species_data[~occurence_species_data['eventDate'].isna()].reset_index(drop=True) # Removing data that does not contain date information
occurence_species_data.loc[occurence_species_data['individualCount'].isna(),'individualCount'] = 1 # Setting 1 as default
occurence_species_data = occurence_species_data[occurence_species_data['decimalLatitude'].notna() & occurence_species_data['decimalLongitude'].notna()] # Removing data without any information about latitude and longitude
occurence_species_data['eventDate'] = pd.to_datetime(occurence_species_data['eventDate'], format="mixed", utc=True) # Setting column as datetime
occurence_species_data.loc[:,'stateProvince'] = occurence_species_data.stateProvince.replace(STATE_NAME_ENCODING) # Renaming

occurence_species_data = occurence_species_data[~occurence_species_data['stateProvince'].isnull()].reset_index(drop=True)

COLUMNS_RENAME = {
    "countryCode":"Pais",
    "locality":"Localizacao",
    "decimalLatitude":"Latitude",
    "decimalLongitude":"Longitude",
    "eventDate":"Data",
    "individualCount":"Contagem de individuos",
    "collectionCode":"Plataforma",
    "stateProvince":"Estado",
    "basisOfRecord":"Fonte do registro"
}

occurence_species_data.rename(columns=COLUMNS_RENAME, inplace=True)
occurence_species_data.to_parquet(GBIF_TREATED_FILE_OUTPUT, index=False)
occurence_species_data['Presence'] = 1
occurence = occurence_species_data[['Latitude','Longitude','Presence']].copy()

occurence.to_parquet(OCCURANCE_FILE, index=False)
occurence_species_data.to_csv("generated_files/gbif_treated.csv", index=False, sep=';')

In [ ]:
print(occurence_species_data.head(2))

del occurence_species_data_unfiltered, occurence_species_data, occurence

## 2 Criando pontos de pseudo-absência

### Definindo o domínio dos pontos
Vamos determinar o domínio como sendo a caixa de recorte que envolve os estados do: [RS, SC, PR, SP, MG, ES, RJ]

Definida por 

$x_{min}, y_{min}, x_{max},y_{max} = (-60, -33.7, -35, -10)$


In [ ]:
from classes.domain import Domain

domain = Domain(
    bounds=(-60,-35,-35,-10),
    output_path="assets/OUTPUT/"
)

occurence_dataframe = pd.read_parquet(OCCURANCE_FILE)

def generate_random_points(bounds: tuple, number: int) -> tuple:
    """
    Generating random points inside boundaries
    """   
    minx, miny, maxx, maxy = bounds
    x = np.random.uniform( minx, maxx, number )
    y = np.random.uniform( miny, maxy, number )
    return x, y

x, y = generate_random_points(domain.bounds, N_PSEUDO_ABSCENCE)

In [ ]:
occurence_dataframe = pd.read_parquet(OCCURANCE_FILE)
geoframe = pd.read_parquet(OCCURANCE_FILE)
geoframe = gpd.GeoDataFrame(geometry=gpd.points_from_xy(geoframe['Longitude'], geoframe['Latitude']))

x, y = generate_random_points(geoframe.total_bounds, N_PSEUDO_ABSCENCE)
gdf_points = gpd.GeoDataFrame(geometry=gpd.points_from_xy(x, y))


#geoframe = gpd.read_file('assets\FEATURES\MALHAS\BR_UF_2022.shp')
#geoframe = geoframe[geoframe['SIGLA_UF'].isin(['MG','SP','RJ','ES','RS','SC])]
#geoframe = geoframe.cx[:-42,:]
# Carregue o shapefile
convex_hull = geoframe.geometry.unary_union.convex_hull
# Use a função contains para filtrar os pontos contidos no shapefile
absence_dataframe = gdf_points[gdf_points.within(convex_hull)]

absence_dataframe['Latitude'] = absence_dataframe['geometry'].y
absence_dataframe['Longitude'] = absence_dataframe['geometry'].x
absence_dataframe['Presence'] = 0

absence_dataframe = absence_dataframe[['Latitude','Longitude','Presence']]

dataframe = pd.concat([absence_dataframe, occurence_dataframe])
dataframe.to_parquet(OCCURANCE_ABSENCE_FILE, index=False)

In [ ]:
fig = px.scatter_mapbox(
    dataframe,
    lat=dataframe['Latitude'],
    lon=dataframe['Longitude'],
    center={"lat": -16.95, "lon": -47.78},
    mapbox_style="carto-positron",
    color='Presence',
    zoom=3,
    opacity=1
)
fig.show()

In [ ]:
print(dataframe['Presence'].value_counts())

del dataframe, absence_dataframe, occurence_dataframe

## 3 Adicionando as features para os pontos gerados

In [ ]:
OCCURENCE_GEO_JSON = "assets/INPUT/occurences.json"
FEATURES_AND_TREATED_DATA = "assets/INPUT/features.parquet"
explanatory_rasters = [
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_1.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_2.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_7.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_12.tif',
    'assets\FEATURES\VEGETACAO\evergreen_broadleaf_trees.tif',
    'assets\FEATURES\ENHANCED_VEGETATION_INDEX\evenness_01_05_1km_uint16.tif',
    'assets\FEATURES\FLAT_TOPOGRAPGIC_CATEGORICAL\geomflat_1KMperc_GMTEDmd.tif'
]

coordinates = pd.read_parquet(OCCURANCE_ABSENCE_FILE)

coordinates['geometry'] = list(zip(coordinates["Longitude"], coordinates["Latitude"]))
coordinates = coordinates[['Presence','geometry']].copy()
coordinates['geometry'] = coordinates["geometry"].apply(Point)
geo_dataframe = gpd.GeoDataFrame(coordinates)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    geo_dataframe,
    crs = {'init': 'epsg:4326'},
    geometry = geo_dataframe['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)
OSD_geoframe.to_file(OCCURENCE_GEO_JSON, driver="GeoJSON")

train_xs, train_y = load_training_vector(OCCURENCE_GEO_JSON, explanatory_rasters, response_field="Presence")

In [ ]:

df = pd.DataFrame(train_xs)
df.loc[:,'Presence'] = train_y
df = df[~df[0].isnull()]
df.columns = ['Anual_Mean_Temp', 'Mean_Diurnal_Range', 'Temperature_Anual_Range', 'Anual_Preciptation','Broadleaf_Trees', 'Eveness', 'Geomflat', 'Presence']
df.to_parquet(FEATURES_AND_TREATED_DATA, index=False)

del coordinates, OSD_geoframe, df

## 4. Criando um modelo

### Maxent

Etapas:
1. Separar os dados em conjunto de treino e testes
2. Ajustar o modelo
3. Avaliar

In [ ]:
# ML 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn import model_selection 
from sklearn import metrics
import matplotlib.pyplot as plt

FEATURES_AND_TREATED_DATA = "assets/INPUT/features.parquet"

# ROC 
def plot_roc_curve(fper, tper):
    plt.plot(fper, tper, color='red', label='ROC')
    plt.plot([0, 1], [0, 1], color='green', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic Curve')
    plt.legend()
    plt.show()
    
# Classifier evaluation functions (modify pyimpute function)
def evaluate_clf(
    clf, X, y, name, k=None, test_size=0.2, scoring="f1_weighted", feature_names=None
):
    print(name)
    X_train, X_test, y_train, y_true = model_selection.train_test_split(
        X, y,
        test_size=test_size, # Test data size
        shuffle=True, # Shuffle the data before split
        stratify=y # Keeping the appearance/non-appearance ratio of Y
    )

    if k: # Cross-validation
        kf = model_selection.KFold(n_splits=k) # k-fold
        scores = model_selection.cross_val_score(clf, X_train, y_train, cv=kf, scoring=scoring)
        print(name + " %d-fold Cross Validation Accuracy: %0.2f (+/- %0.2f)"
              % (k, scores.mean() * 100, scores.std() * 200))
        print()
    
    clf.fit(X_train, y_train) # Training of classifiers
    y_pred = clf.predict(X_test) # Classifier predictions
    
    # Classifier evaluation metrics
    print("Accuracy Score: %.2f" % metrics.accuracy_score(y_true, y_pred))
    print()

    print("Classification report")
    print(metrics.classification_report(y_true, y_pred))
    print()

    print("Confussion matrix")
    print(metrics.confusion_matrix(y_true, y_pred))
    print()
    
    print('AUC(ROC): %.2f'% metrics.roc_auc_score(y_true, y_pred))
    print()
       
    # ROC 
    probs = clf.predict_proba(X_test) 
    prob = probs[:, 1]  
    fper, tper, thresholds = metrics.roc_curve(y_true, prob)
    plot_roc_curve(fper, tper)

    if hasattr(clf, "feature_importances_"):
        print("Feature importances")
        for f, imp in zip(feature_names, clf.feature_importances_):
            print("%20s: %s" % (f, round(imp * 100, 1)))
        print()
    return clf
FEATURES_AND_TREATED_DATA = "assets/INPUT/features.parquet"
df = pd.read_parquet(FEATURES_AND_TREATED_DATA)
df = df.dropna()

y = df.pop("Presence")
X = df.copy()

In [ ]:
maxent = LogisticRegression(max_iter=1_000)
maxent = evaluate_clf(maxent, X, y, "MaxEnt", k=5, test_size=0.2, scoring="f1_weighted", feature_names=X.columns)

In [ ]:
X.columns

In [ ]:
random_forest = RandomForestClassifier()
random_forest = evaluate_clf(random_forest, X, y, "Random Forest", k=5, test_size=0.2, scoring="f1_weighted", feature_names=X.columns)

## 5. Predição

In [ ]:
from pathlib import Path

geoframe = gpd.read_file('assets\FEATURES\MALHAS\BR_UF_2022.shp')
geoframe = geoframe[geoframe['SIGLA_UF'].isin(['MG','SP','RJ','ES'])]
geoframe = geoframe.cx[:-42,:]

OCCURENCE_GEO_JSON_PREDICT = "assets/INPUT/predict.json"
FEATURES_AND_TREATED_DATA_PREDICT = "assets/INPUT/features_predict.parquet"

domain_saida = Domain(
    bounds=(-46,-25,-39,-18),
    output_path="assets/MASKED_FEATURES_SHP/"
)

explanatory_rasters = [
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_1.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_2.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_7.tif',
    'assets\FEATURES\TEMPERATURE\wc2.1_30s_bio_12.tif',
    'assets\FEATURES\VEGETACAO\evergreen_broadleaf_trees.tif',
    'assets\FEATURES\ENHANCED_VEGETATION_INDEX\evenness_01_05_1km_uint16.tif',
    'assets\FEATURES\FLAT_TOPOGRAPGIC_CATEGORICAL\geomflat_1KMperc_GMTEDmd.tif'
]

new_explanatory_rasters = [
    'assets/MASKED_FEATURES_SHP/wc2.1_30s_bio_1.tif',
    'assets/MASKED_FEATURES_SHP/wc2.1_30s_bio_2.tif',
    'assets/MASKED_FEATURES_SHP/wc2.1_30s_bio_7.tif',
    'assets/MASKED_FEATURES_SHP/wc2.1_30s_bio_12.tif',
    'assets/MASKED_FEATURES_SHP/evergreen_broadleaf_trees.tif',
    'assets/MASKED_FEATURES_SHP/evenness_01_05_1km_uint16.tif',
    'assets/MASKED_FEATURES_SHP/geomflat_1KMperc_GMTEDmd.tif'
]

In [ ]:
for file in explanatory_rasters:
    domain_saida.mask_tiff_with_shapefile(tiff_file=file,geoframe = geoframe, output_file_name=Path(file).name)
    
target_xs, raster_info = pyimpute.load_targets(new_explanatory_rasters)

In [ ]:
pyimpute.impute(target_xs, maxent, raster_info, outdir='OUTPUT/' + 'MAXENT' + '-IMAGES', class_prob=True, certainty=True)
pyimpute.impute(target_xs, random_forest, raster_info, outdir='OUTPUT/' + 'RF' + '-IMAGES', class_prob=True, certainty=True)

In [ ]:
def plotit(x, title, cmap="Blues"):
    plt.figure(figsize = (14,7))
    plt.imshow(x, cmap=cmap, interpolation='nearest')
    plt.colorbar()
    plt.title(title, fontweight = 'bold')
    
distr_rf = rasterio.open("OUTPUT\MAXENT-IMAGES\probability_1.tif").read(1) 
distr_et = rasterio.open("OUTPUT\RF-IMAGES\probability_1.tif").read(1)

distr_averaged = (distr_rf + distr_et)/2

In [ ]:
from shapely.geometry import Polygon

def calculate_tiff_mean(tiff_file, minx, miny, maxx, maxy):
    # Abrir o arquivo .tiff
    with rasterio.open(tiff_file) as src:
        # Recortar o raster com a geometria do polígono
        bbox = [{
            'type': 'Polygon',
            'coordinates': [[
                (minx, miny),
                (minx, maxy),
                (maxx, maxy),
                (maxx, miny),
                (minx, miny)
            ]]
        }]
        try:
            out_image, out_transform = rasterio.mask.mask(src, bbox, crop=True, nodata=np.nan)
            # Calcular a média dos valores dentro do polígono
            mean_value = np.nanmean(out_image)
        except:
            return 0
    return mean_value

def create_grid_with_contours(gdf, n_cols, n_rows, tiff_file):
    # Obter limites do shapefile
    xmin, ymin, xmax, ymax = gdf.total_bounds
    #xmin, ymin, xmax, ymax = [-53.11011153, -41, -28.84763991, -14.23318067]
    # Calcular o tamanho dos retângulos
    dx = (xmax - xmin) / n_cols
    dy = (ymax - ymin) / n_rows

    # Lista para armazenar polígonos
    polygons = []
    mean_val = []
    x_min = []
    x_max = []
    y_min = []
    y_max = []
    # Loop para criar os polígonos
    for i in range(n_rows):
        for j in range(n_cols):
            x0 = xmin + j * dx
            y0 = ymin + i * dy
            x1 = x0 + dx
            y1 = y0 + dy
            
            x_min.append(x0)
            y_min.append(y0)
            x_max.append(x1)
            y_max.append(y1)
            polygons.append(Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)]))
            mean_val.append(calculate_tiff_mean(tiff_file,x0, y0, x1, y1))
    # Criar GeoDataFrame a partir dos polígonos
    grid_gdf = gpd.GeoDataFrame(geometry=polygons, crs=gdf.crs)
    grid_gdf['x_min'] = x_min
    grid_gdf['y_min'] = y_min
    grid_gdf['x_max'] = x_max
    grid_gdf['y_max'] = y_max
    grid_gdf['mean_val'] = mean_val
    # Interseção com o shapefile original
    grid_gdf = gpd.overlay(grid_gdf, gdf, how='intersection')

    return grid_gdf

# Caminho para o arquivo .shp
tiff_file = 'OUTPUT\MAXENT-IMAGES\probability_1.tif'

# Número de colunas e linhas na malha
n_cols = 50
n_rows = 50

# Criar a malha quadriculada com contornos
grid_with_contours = create_grid_with_contours(geoframe.cx[:-42,:], n_cols, n_rows, tiff_file)

In [ ]:
# Plotar a malha quadriculada com contornos
grid_with_contours.plot(column='mean_val', edgecolor='black', cmap='viridis', legend=True)

In [ ]:

# Plotar o heatmap com Plotly Express
fig = px.choropleth_mapbox(grid_with_contours, geojson=grid_with_contours.geometry.__geo_interface__,
                           locations=grid_with_contours.index, color='mean_val',
                           color_continuous_scale="viridis",
                           range_color=(0, 1),
                           center={"lat": -16.95, "lon": -47.78},
                           mapbox_style="carto-positron",
                           zoom=3,
                           opacity=1,
                           labels={'mean_val': 'Mean Value'}
                          )
# Adicionar contorno dos polígonos
OCCURANCE_FILE = "assets/INPUT/occurence.parquet"
occurence_df = pd.read_parquet(OCCURANCE_FILE)
with open("assets/INPUT/occurences.json") as jfile:
    geo_json = json.load(jfile)
fig.update_traces(marker_line_width=0)
fig.add_trace(go.Scattermapbox(
        lat=occurence_df['Latitude'],
        lon=occurence_df['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color='rgb(0, 0, 0)',
            opacity=1
        ),
        hoverinfo='none'
    ))

fig.add_trace(go.Scattermapbox(
        lat=occurence_df['Latitude'],
        lon=occurence_df['Longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=6,
            color='rgb(194, 33, 36)',
            opacity=0.7
        ),
        hoverinfo='none'
))

fig.update_geos(
    visible=False,
    resolution=50,
    lataxis_gridcolor="red",
    lataxis_showgrid=True,
    lataxis_dtick=15,
    lonaxis_showgrid=True,
    lonaxis_dtick=15,
)
fig.show()

In [ ]:
plotit(distr_averaged, "Species Suitability", cmap="Blues")

## 6. PCA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

In [ ]:
FEATURES_AND_TREATED_DATA = "assets/INPUT/features.parquet"
from sklearn.feature_selection import mutual_info_regression

df = pd.read_parquet(FEATURES_AND_TREATED_DATA)
df = df.dropna()

y = df.pop("Presence")
X = df.copy()

def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings

def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs


def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores

pca, X_pca, loadings = apply_pca(X)

In [ ]:
plot_variance(pca)

In [ ]:
mi_scores=make_mi_scores(X, y)
fig = go.Figure([go.Bar(
    x=mi_scores.index, y=mi_scores.values
)])
fig.update_layout(
    
)
fig.show()

## Validando multicolinearidade

## Análise de multicolinearidade

Analisando se os parâmetros são os mesmos nas condições de fitar o modelo com ou sem uma das variáveis.

Se eles são não correlacionados, então os coeficientes devem ser iguais para o caso em que ambos estão x quando X1 está e quando ambos estão x quando X2 está

$$
SSR(X_1|X_2) = SSR(X_1) = SSE(X_2) - SSE(X_1,X_2)
$$
SSR(X)

##
Quando existem duas variáveis independentes correlacionadas, os coeficientes da regressão dependem se um ou outra variável está inclusida no modelo

- $SSR(X_1)$: Redução de variação da variável resposta ao introduzir a variável $X_1$.
- $SSR(X_1|X_2)$: Corresponde a redução de variação da resposta ao introduzir a variável $X_1$ dado que a $X_2$ já está no modelo